In [4]:
import sys
import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.4.1
Keras Version: 2.4.0

Python 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
Pandas 1.1.3
Scikit-Learn 0.23.2
GPU is NOT AVAILABLE


In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS
import nltk
import re
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from tensorflow.keras.preprocessing.text import one_hot, Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Embedding, Input, LSTM, Conv1D, MaxPool1D, Bidirectional
from tensorflow.keras.models import Model
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ifyuc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
import numpy as np
import pandas as pd
import itertools
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics
from sklearn.metrics import plot_confusion_matrix
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

In [7]:
df=pd.read_csv('train.csv')
df = shuffle(df)
df = df.reset_index(drop=True)
df.isnull().sum()    
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18285 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      18285 non-null  int64 
 1   title   18285 non-null  object
 2   author  18285 non-null  object
 3   text    18285 non-null  object
 4   label   18285 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 857.1+ KB


In [8]:
'''conversion_dict = { 0:'Real', 1:'Fake'}
df['label'] = df['label'].replace(conversion_dict)
df.label.value_counts()'''

"conversion_dict = { 0:'Real', 1:'Fake'}\ndf['label'] = df['label'].replace(conversion_dict)\ndf.label.value_counts()"

In [9]:
df.drop(["title"],axis=1,inplace=True)
df.drop(["id"],axis=1,inplace=True)
df.drop(["author"],axis=1,inplace=True)

df['text']=df['text'].apply(str)
df['text'] = df['text'].apply(lambda x: x.lower())

df = shuffle(df)
df = df.reset_index(drop=True)

In [10]:
df['text'][0]

'  national highway traffic safety administration in the usa today gave clarification on the soon to be imposed safety measures required for electric cars. in addition to a two-tone siren that must sound the word “pus-sy!” and a yellow strobe light mounted on the roof, all owners must carry a man who will walk in front of the car holding a red flag. \n‘the nhtsa has worked long and hard on these proposals with our colleagues in detroit,’ a spokesman – george mustangveight – said. ‘over one americans every year are injured by electric cars and we aim to resolve this unacceptable risk to our citizens with these actions. plus, the forced employment of a pedestrian speed retarder by all owners of these dangerous vehicles will do wonders for the unemployment these blights have caused in our oil and petrochemical industries.’ \nin california, tesla motors owner elon musk remained positive: ‘it’s fine. i’m currently working on a way to convert electricity using the phone lines as a transport 

In [11]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ifyuc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [13]:
#2 and 1 letter words delete and return list
def remove_stopwords(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in stop_words:
            result.append(token)
            
    return result

In [14]:
df['text'] = df['text'].apply(remove_stopwords)

In [16]:
df['text'][0]

['national',
 'highway',
 'traffic',
 'safety',
 'administration',
 'today',
 'gave',
 'clarification',
 'soon',
 'imposed',
 'safety',
 'measures',
 'required',
 'electric',
 'cars',
 'addition',
 'tone',
 'siren',
 'sound',
 'word',
 'yellow',
 'strobe',
 'light',
 'mounted',
 'roof',
 'owners',
 'carry',
 'walk',
 'holding',
 'flag',
 'nhtsa',
 'worked',
 'long',
 'hard',
 'proposals',
 'colleagues',
 'detroit',
 'spokesman',
 'george',
 'mustangveight',
 'said',
 'americans',
 'year',
 'injured',
 'electric',
 'cars',
 'resolve',
 'unacceptable',
 'risk',
 'citizens',
 'actions',
 'plus',
 'forced',
 'employment',
 'pedestrian',
 'speed',
 'retarder',
 'owners',
 'dangerous',
 'vehicles',
 'wonders',
 'unemployment',
 'blights',
 'caused',
 'petrochemical',
 'industries',
 'california',
 'tesla',
 'motors',
 'owner',
 'elon',
 'musk',
 'remained',
 'positive',
 'fine',
 'currently',
 'working',
 'convert',
 'electricity',
 'phone',
 'lines',
 'transport',
 'medium',
 'sure',
 'runn

In [17]:
df

,text,label
0,"[national, highway, traffic, safety, administr...",1
1,"[want, support, receive, access, tons, bonus, ...",1
2,"[kabul, afghanistan, months, failed, pakistani...",0
3,"[tatiana, ricardo, single, mother, children, i...",0
4,"[tuesday, news, channel, hannity, mark, meadow...",0
...,...,...
18280,"[janeiro, famous, athletes, world, like, muham...",0
18281,"[david, duke, prof, kevin, macdonald, duke, ov...",1
18282,"[rufus, farmer, tired, ways, black, mistreated...",0
18283,"[powerful, array, republican, party, largest, ...",0


In [18]:
list_of_words = []
for i in df.text:
    for j in i:
        list_of_words.append(j)

In [19]:
len(list_of_words)

6776215

In [20]:
total_words = len(list(set(list_of_words)))
total_words #unique words

149278

In [21]:
df['text_of_use'] = df['text'].apply(lambda x: " ".join(x))

In [22]:
df['text_of_use'][0]

'national highway traffic safety administration today gave clarification soon imposed safety measures required electric cars addition tone siren sound word yellow strobe light mounted roof owners carry walk holding flag nhtsa worked long hard proposals colleagues detroit spokesman george mustangveight said americans year injured electric cars resolve unacceptable risk citizens actions plus forced employment pedestrian speed retarder owners dangerous vehicles wonders unemployment blights caused petrochemical industries california tesla motors owner elon musk remained positive fine currently working convert electricity phone lines transport medium sure running beta failing soon able intolerable smugness owners pure energy james_doc'

In [23]:
df['text'][0]

['national',
 'highway',
 'traffic',
 'safety',
 'administration',
 'today',
 'gave',
 'clarification',
 'soon',
 'imposed',
 'safety',
 'measures',
 'required',
 'electric',
 'cars',
 'addition',
 'tone',
 'siren',
 'sound',
 'word',
 'yellow',
 'strobe',
 'light',
 'mounted',
 'roof',
 'owners',
 'carry',
 'walk',
 'holding',
 'flag',
 'nhtsa',
 'worked',
 'long',
 'hard',
 'proposals',
 'colleagues',
 'detroit',
 'spokesman',
 'george',
 'mustangveight',
 'said',
 'americans',
 'year',
 'injured',
 'electric',
 'cars',
 'resolve',
 'unacceptable',
 'risk',
 'citizens',
 'actions',
 'plus',
 'forced',
 'employment',
 'pedestrian',
 'speed',
 'retarder',
 'owners',
 'dangerous',
 'vehicles',
 'wonders',
 'unemployment',
 'blights',
 'caused',
 'petrochemical',
 'industries',
 'california',
 'tesla',
 'motors',
 'owner',
 'elon',
 'musk',
 'remained',
 'positive',
 'fine',
 'currently',
 'working',
 'convert',
 'electricity',
 'phone',
 'lines',
 'transport',
 'medium',
 'sure',
 'runn

In [24]:
df

,text,label,text_of_use
0,"[national, highway, traffic, safety, administr...",1,national highway traffic safety administration...
1,"[want, support, receive, access, tons, bonus, ...",1,want support receive access tons bonus content...
2,"[kabul, afghanistan, months, failed, pakistani...",0,kabul afghanistan months failed pakistani effo...
3,"[tatiana, ricardo, single, mother, children, i...",0,tatiana ricardo single mother children includi...
4,"[tuesday, news, channel, hannity, mark, meadow...",0,tuesday news channel hannity mark meadows said...
...,...,...,...
18280,"[janeiro, famous, athletes, world, like, muham...",0,janeiro famous athletes world like muhammad ti...
18281,"[david, duke, prof, kevin, macdonald, duke, ov...",1,david duke prof kevin macdonald duke overwhelm...
18282,"[rufus, farmer, tired, ways, black, mistreated...",0,rufus farmer tired ways black mistreated natio...
18283,"[powerful, array, republican, party, largest, ...",0,powerful array republican party largest financ...


In [25]:
maxlen = -1
for doc in df.text_of_use:
    tokens = nltk.word_tokenize(doc)
    if(maxlen<len(tokens)):
        maxlen = len(tokens)
print("text of use da en cok kelimeye sahip single row su kadar kelimeye sahip", maxlen) #word embeding icin

text of use da en cok kelimeye sahip single row su kadar kelimeye sahip 13682


In [26]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df.text_of_use, df.label, test_size = 0.2)

In [27]:
from nltk import word_tokenize

In [42]:
y_test[0]

1

In [28]:
tokenizer = Tokenizer(num_words = total_words) #tokenizer create to token unique words
tokenizer.fit_on_texts(x_train)
train_sequences = tokenizer.texts_to_sequences(x_train)
test_sequences = tokenizer.texts_to_sequences(x_test)

In [44]:
test_sequences[0]

[70,
 2914,
 1511,
 17,
 211,
 418,
 13143,
 2181,
 470,
 1825,
 438,
 11988,
 3030,
 457,
 10540,
 5566,
 3297,
 8481,
 133,
 211,
 418,
 1680,
 4289,
 3244,
 11864,
 17638,
 406,
 1230,
 247,
 19859,
 30465,
 46077,
 416,
 780,
 12805,
 2945,
 6327,
 28337,
 17483,
 406,
 165,
 432,
 25976,
 406,
 1230,
 5965,
 28337,
 17483,
 7412,
 1891,
 4395,
 32810,
 13143,
 2181,
 470,
 1825,
 438,
 11988,
 3030,
 457,
 10540,
 5566,
 3297,
 8481,
 133,
 359,
 1259,
 105,
 15370,
 4,
 1717,
 12429,
 65,
 3981,
 11245,
 1280,
 922,
 1245,
 4822,
 161,
 3395,
 2474,
 2681,
 21048,
 3285,
 5322,
 4339,
 239,
 9048,
 6371,
 2126,
 150,
 30144,
 753,
 11145,
 64,
 13143,
 2181,
 358,
 2729,
 61,
 450,
 498,
 6726,
 13143,
 1055,
 11,
 954,
 752,
 3825,
 752,
 740,
 203,
 522,
 13143,
 638,
 878,
 96,
 409,
 4728,
 578,
 1977,
 11,
 4784,
 96,
 954,
 37487,
 1160,
 5896,
 813,
 4867,
 1598,
 4784,
 2711,
 924,
 80,
 3328,
 3849,
 117,
 985,
 892,
 3223,
 1267,
 11,
 740,
 224,
 1416,
 985,
 8308,
 31

In [29]:
df.text_of_use[10]

'hong kong taiwan scrambled fighter jets dispatched frigate taiwan strait wednesday china sent sole aircraft carrier waterway taiwan official central news agency reported transit aircraft carrier liaoning came amid rising tensions taiwan china donald trump broke decades protocol speaking phone taiwan president tsai election victory tsai leads political party traditionally supported taiwan formal independence china tsai visiting central america week calls officials taiwan seeking updates liaoning transit central news agency reported citing alex huang president spokesman china decision send carrier waterway separates taiwan reflects early foreign policy challenge trump force think intended intimidate worrisome taiwan point view know going ratchet pressures tensions said bonnie glaser senior adviser asia center strategic international studies washington trump administration test resolve suspect push pretty forcefully china sent carrier conducting exercises south china taiwan strait wednes

In [30]:
train_sequences[10] #encoding for df.text_of_use[0] ->>> bi sorun var

[490,
 40611,
 1054,
 20801,
 2126,
 822,
 2929,
 1544,
 5209,
 1948,
 2027,
 1084,
 497,
 1276,
 11,
 19669,
 44866,
 24530,
 77810,
 2044,
 179]

In [37]:
test_sequences[10]

[10524,
 9705,
 1039,
 6,
 32,
 2,
 268,
 272,
 1290,
 2104,
 585,
 2,
 153,
 2104,
 187,
 4022,
 12517,
 5893,
 14,
 16736,
 850,
 2200,
 656,
 418,
 1975,
 548,
 6,
 2272,
 1608,
 10759,
 59,
 1117,
 576,
 315,
 9915,
 6,
 700,
 15,
 22486,
 625,
 3363,
 31,
 9705,
 843,
 5644,
 43,
 6,
 464,
 4435,
 1428,
 162,
 2670,
 9705,
 7906,
 15962,
 2,
 442,
 129,
 10524,
 1243,
 323,
 709,
 3309,
 589,
 7044,
 2706,
 6,
 9705,
 1039,
 48,
 2028,
 622,
 549,
 56041,
 3478,
 7051,
 1608,
 2644,
 2059,
 1820,
 9705,
 1468,
 2644,
 2059,
 2753,
 33,
 4857,
 3337,
 10249,
 559,
 1859,
 2,
 1257,
 323,
 4596,
 1339,
 3836,
 1396,
 5280,
 13181,
 585,
 6,
 6185,
 589,
 323,
 400,
 5012,
 23427,
 593,
 6,
 1056,
 11363,
 2519,
 24834,
 40062,
 861,
 99,
 2,
 153,
 171,
 24655,
 2722,
 4436,
 5980,
 12727,
 55,
 743,
 1150,
 385,
 171,
 1023,
 560,
 488,
 7280,
 171,
 1023,
 24655,
 2722,
 5614,
 2,
 550,
 1150,
 422,
 410,
 3869,
 662,
 1013,
 8091,
 1538,
 6692,
 2104,
 4471,
 724,
 5533,
 2281,
 

In [31]:
padded_train = pad_sequences(train_sequences,maxlen = 40, padding = 'post', truncating = 'post')
padded_test = pad_sequences(test_sequences,maxlen = 40, truncating = 'post') 

In [35]:
'''for i,doc in enumerate(padded_train[:2]):
     print("The padded encoding for document",i+1," is : ",doc)'''
padded_train[0] #padded single text

array([  717,   194,  1968,  5307,   789,    14,    17,  5513,   323,
        6787, 34640,  1364,  1690, 16338,   525,   143,  1011,   369,
       16338,   667,  1237, 16338,  1924,  1011,   877,  3056,   659,
       19661,  1104,  6666,  7984,  1146,   429,  5893,    38,  4953,
          33, 25434,  1205,  3929])

In [36]:
padded_test[0] #padded single label

array([   70,  2914,  1511,    17,   211,   418, 13143,  2181,   470,
        1825,   438, 11988,  3030,   457, 10540,  5566,  3297,  8481,
         133,   211,   418,  1680,  4289,  3244, 11864, 17638,   406,
        1230,   247, 19859, 30465, 46077,   416,   780, 12805,  2945,
        6327, 28337, 17483,   406])

In [29]:
# Sequential Model
model = Sequential()

# Embeddidng layer
model.add(Embedding(total_words, output_dim = 128))


# Bi-Directional RNN and LSTM
model.add(Bidirectional(LSTM(128)))

# Dense layers
model.add(Dense(128, activation = 'relu'))
model.add(Dense(1,activation= 'sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         19107584  
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               263168    
_________________________________________________________________
dense (Dense)                (None, 128)               32896     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 19,403,777
Trainable params: 19,403,777
Non-trainable params: 0
_________________________________________________________________


In [30]:
y_train = np.asarray(y_train)

In [31]:
model.fit(padded_train, y_train, batch_size = 64, validation_split = 0.1, epochs = 5)

Epoch 1/5
206/206 [==============================] - 61s 279ms/step - loss: 0.4132 - acc: 0.7906 - val_loss: 0.1846 - val_acc: 0.9214
Epoch 2/5
206/206 [==============================] - 57s 275ms/step - loss: 0.0602 - acc: 0.9797 - val_loss: 0.2028 - val_acc: 0.9214
Epoch 3/5
206/206 [==============================] - 57s 277ms/step - loss: 0.0157 - acc: 0.9963 - val_loss: 0.3292 - val_acc: 0.9111
Epoch 4/5
206/206 [==============================] - 56s 274ms/step - loss: 0.0041 - acc: 0.9985 - val_loss: 0.3051 - val_acc: 0.9200
Epoch 5/5
206/206 [==============================] - 56s 272ms/step - loss: 0.0063 - acc: 0.9984 - val_loss: 0.4983 - val_acc: 0.9139


In [32]:
pred = model.predict(padded_test)

In [33]:
prediction = []
for i in range(len(pred)):
    if pred[i].item() > 0.95:
        prediction.append(1)
    else:
        prediction.append(0)

In [34]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(list(y_test), prediction)

print("Model Accuracy : ", accuracy)

Model Accuracy :  0.9012852064533771


In [35]:
confusion_matrix(y_test,prediction, labels=[0,1]) #1608 true tested text which 63 times ML was wrong 

array([[1946,  143],
       [ 218, 1350]], dtype=int64)

In [36]:
import os.path
if os.path.isfile('models/lstm_model.h5') is False:
    model.save('models/lstm_model.h5')